In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
from IPython.display import clear_output
import tensorflow as tf
import pandas as pd

In [2]:
# Load dataset
cols = ['sepal_len', 'sepal_width', 'petal_len', 'petal_width', 'species']
train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

# Get X
train = pd.read_csv(train_path, names=cols, header=0)
test = pd.read_csv(test_path, names=cols, header=0)

# Get y
train_y = train.pop('species')
test_y = test.pop('species')
print(train.head())

   sepal_len  sepal_width  petal_len  petal_width
0        6.4          2.8        5.6          2.2
1        5.0          2.3        3.3          1.0
2        4.9          2.5        4.5          1.7
3        4.9          3.1        1.5          0.1
4        5.7          3.8        1.7          0.3


In [3]:
# Input function
def input_fn(features, labels, training=True, batch_size=256):
    # Convert inputs to a tf dataset
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    
    if training:
        dataset = dataset.shuffle(1000).repeat()

    return dataset.batch(batch_size)

In [4]:
# Create list of tf feature columns
feature_cols = [tf.feature_column.numeric_column(key=key) for key in train.keys()]

feature_cols

[NumericColumn(key='sepal_len', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='sepal_width', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='petal_len', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='petal_width', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [10]:
# DNN with 2 hidden layers - 30 and 10 hidden nodes respectively
model = tf.estimator.DNNClassifier(
    feature_columns=feature_cols,
    hidden_units=[30, 10],
    n_classes=3)

# Train model
model.train(
    input_fn=lambda: input_fn(train, train_y, training=True),
    steps=500)
clear_output()

In [11]:
# Evaluate on test set
eval_result = model.evaluate(
    input_fn=lambda: input_fn(test, test_y, training=False))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-01-05T21:45:29Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpw_2eonj2/model.ckpt-500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.22766s
INFO:tensorflow:Finished evaluation at 2021-01-05-21:45:30
INFO:tensorflow:Saving dict for global step 500: accuracy = 0.26666668, average_loss = 1.0383085, global_step = 500, loss = 1.0383085
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 500: /tmp/tmpw_2eonj2/model.ckpt-500

Test set accuracy: 0.267



In [ ]:
# Why is accuracy sometimes 0.533 and sometimes 0.933?